<a href="https://colab.research.google.com/github/BenjaminWinters465/my_module_5/blob/main/lab_5_2_scraping_imdb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 5.2 -- Scraping IMBD

Our goal is to scrap [IMDB](imdb.com) user reviews for *Borat Subsequent Moviefilm*.  Unfortunately, the page for user reviews only shows a limited number of reviews and you can't access additional pages through a link.  `selenium` to the rescue! In this lab, we will combine our two approaches to web scraping by

1. Using `selenium` to load the page and click the *Load More* until we have all the reviews.
2. Creating a `BeautifulSoup` instance for the complete page and parsing the results.

### Task 1 -- Load the reviews.

Explore IMBD to find the web link for the user reviews for *Borat Subsequent Moviefilm* and load this page in Python with `selenium`.

In [60]:
# For running in colab
!pip install kora -q

In [61]:
# For running in colab
url = 'https://www.imdb.com/title/tt13143964/reviews?ref_=tt_ql_3'
from kora.selenium import wd as driver
driver.get(url)

### Task 2 -- Figure out how to click the *Load More* button.

To load all of the user reviews, we need to click the *Load More* button multiple times.  First, find the corresponding WebElement and verify that clicking this button loads another page of results.

In [54]:
driver.find_element_by_class_name('load-more-data').click()

### Task 3 -- Click *Load More* until you have all the results.

Now you need to write code that will keep clicking the *Load More* button when you find it.  **Hint:** We can think of this as an example of an *unfold* process, meaning you should use a `while` loop combined with a [try-and-except statement](https://pythonbasics.org/try-except/) to keep trying to click the button.  To make sure you don't get an infinite loop, use a variable to identify and hold the stopping condition/state.

In [62]:
while True:
  try:
    driver.find_element_by_class_name('load-more-data').click()
  except Exception:
    print('End of Page')
    break

print('Should reach here')

End of Page
Should reach here


### Task 4 -- Load the results in a `BeautifulSoup` object.

Since `bs4` has better tools for parsing html, we will now switch to using this module to parse the results.  Recall that you can access the content of the current content from the `selenium` driver using `driver.page_source`.  You can use this attribute to make a `soup` object for the page using 

> soup = BeautifulSoup(driver.page_source, 'html.parser')

In [63]:
import requests
from bs4 import BeautifulSoup

s = requests.Session()
r = s.get('https://www.imdb.com/title/tt13143964/reviews?ref_=tt_ql_3')
imdb_bor = BeautifulSoup(driver.page_source, 'html.parser')

### Task 5 -- Extract the information

Now extract the following data to a csv file.

1. Title
2. Score
3. User
4. Date
5. Text (replace commas with semi-colons!)
6. Two columns for X and Y, where `"X out of Y found this helpful"`
7. Permanent link the the review.


In [66]:
!pip install composablesoup

     |████████████████████████████████| 122kB 6.0MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [67]:
# Imports
from composablesoup import find, find_all, get_text, has_attr
from composable.sequence import slice, head, to_list
from composable.strict import map, filter
from composable.string import replace, strip, split
from composable import from_toolz as tlz
import requests
from bs4 import BeautifulSoup
from composable import pipeable
import re
from composablesoup.soup import find_parent, parents, children, find_previous_sibling
from composablesoup.soup import find_previous_sibling, find_previous_siblings, find_next_sibling, find_next_siblings

In [73]:
(imdb_bor
 >> find('div', attrs = {'class':'review-container'})
 >> find('a', attrs = {'class':'title'})
 >> get_text
 >> strip
)

'Borat Make a Number 2'

In [81]:
(imdb_bor
 >> find('div', attrs = {'class':'review-container'})
 >> find('span', attrs = {'class':'point-scale'})
 >> find_previous_sibling
 >> get_text

)

'10'

In [109]:
user = re.compile('/user/ur\d+/\?ref_=tt_urv')
(imdb_bor
 >> find('div', attrs = {'class':'review-container'})
 >> find('a', attrs = {'href':user})
 >> get_text

)

'MissCzarChasm'

In [89]:
(imdb_bor
 >> find('div', attrs = {'class':'review-container'})
 >> find('span', attrs = {'class':'review-date'})
 >> get_text

)

'29 October 2020'

In [145]:
#interesting interaction where double line break leaves no space between the period of the previous line and first letter of next line
nospace = re.compile('\.(?! )')
(imdb_bor
 >> find('div', attrs = {'class':'review-container'})
 >> find('div', attrs = {'class':'text'})
 >> get_text
 >> replace(',',';')
)

'Borat Make a *Glorious* #2! Subsequent Moviefilm: Delivery of Prodigious Bribe to American Regime for Make Benefit Once Glorious Nation of Kazakhstan is very naiiice!America Mayor Rudolph Giuliani say he not like film.America Mayor Rudolph Giuliani say he very much LIE\ndown to fix pants like in nation of Kazakhstan where we not stand up to tuck the shirt. Much success.You watch.Chin qui'

In [169]:
get_numbs = pipeable(lambda soup: [int(word) for word in soup.split() if word.isdigit()])
get_first = pipeable(lambda soup: [item[0] for item in soup])
#find('div', attrs = {'class':'actions text-muted'})
#get_textNone = pipeable(lambda soup: [entry.get_text if entry.get_text is not None else entry for entry in soup])
(imdb_bor
 >> find_all('div', attrs = {'class':'review-container'})
 >> map(find('div', attrs = {'class':'actions text-muted'}))
 #>> map(get_textNone)

 #>> map(get_text)
 #>> map(get_numbs)
 #>> map([0])
)

[<div class="actions text-muted">
                     162 out of 237 found this helpful.
                         <span>
                             Was this review helpful? <a href="/registration/signin?ref_=urv"> Sign in</a> to vote.
                         </span>
 <br/>
 <a href="/review/rw6217081/?ref_=tt_urv">Permalink</a>
 </div>, None, <div class="actions text-muted">
                     136 out of 200 found this helpful.
                         <span>
                             Was this review helpful? <a href="/registration/signin?ref_=urv"> Sign in</a> to vote.
                         </span>
 <br/>
 <a href="/review/rw6213611/?ref_=tt_urv">Permalink</a>
 </div>, <div class="actions text-muted">
                     187 out of 288 found this helpful.
                         <span>
                             Was this review helpful? <a href="/registration/signin?ref_=urv"> Sign in</a> to vote.
                         </span>
 <br/>
 <a href="/review/rw6210276/?ref